In [15]:
%pip install libmagic

  Preparing metadata (setup.py) ... done
  Created wheel for libmagic: filename=libmagic-1.0-py3-none-any.whl size=4312 sha256=887b04b3117215646f8481f4394f9a0f0c083efecbc4c513e5ce37f55bf72051
  Stored in directory: /Users/natalieyang/Library/Caches/pip/wheels/3e/35/13/53c696ff5f3523479505ec5ed4156198671fb598fe678178bf
Successfully built libmagic
Note: you may need to restart the kernel to use updated packages.


In [24]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import pipeline
from langchain_text_splitters import RecursiveCharacterTextSplitter
import torch
from langchain_community.document_loaders import DirectoryLoader


In [16]:
file_path = 'raw_data/description_pages'
loader = DirectoryLoader(file_path,
                                     glob='*.txt',
                                     show_progress=True,
                                     use_multithreading=True)
documents = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
100%|██████████| 3/3 [00:00<00:00,  5.79it/s]


In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1024, chunk_overlap = 100)
chunks = [chunk for doc in documents
                  for chunk in text_splitter.split_documents([doc])]


In [23]:
chunks_content = [chunk.page_content for chunk in chunks]

In [ ]:
#need to sample the chunks for context in the future
len(chunks_content)

759

In [ ]:
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1")

In [ ]:
fact = ["The first known European to enter the region was the French explorer Robert de La Salle from Quebec during his 1669 expedition down the Ohio River.", "In 1859, the Clinton and Soho iron furnaces introduced coke-fire smelting to the region.", "The NHL's Pittsburgh Penguins have played in Pittsburgh since the team's founding in 1967."]
question = ["Who was the first known person to enter the Pittsburgh?", "When did coke-fire smelting introduced in Pittsburgh?", "When did Penguins funded?"]
answer = ["The first person to enter the Pittsburgh was the French explorer Robert de La Salle", "Coke-fire smelting was introduced in Pittsburgh in 1859 by the Clinton and Soho iron furnaces.","Penguins were funded in 1967."]
#formatted in json and save it
data = []
for i in range(len(fact)):
    data.append({"fact": fact[i], "question": question[i], "answer": answer[i]})
import json
with open('Annotation/example.json', 'w') as f:
    json.dump(data, f)

In [ ]:
def prompt_generation(examples, context, num_questions):
    prompt = "You are an AI assistant trained for data annotation.\n"
    prompt += "Your task is to generate **fact-question-answer pairs** based on the given factual context. You will be given a context passage, and you will select a fact from the context, then ask a question from it, and then provide the answer to the asked question based on the selected fact. "
    prompt += "Ensure the questions are well-formed, unambiguous, and directly answerable using the provided context. Avoid speculative, open-ended questions, or generate any new context. "
    prompt += "These are some examples with questions and answers as well as the fact that help answer that question: \n"
    for i, example in enumerate(examples):
        prompt += f"Fact: {example['fact']}\n"
        prompt += f"Question: {example['question']}\n"
        prompt += f"Answer: {example['answer']}\n"
        prompt += "\n"
    prompt += "Here is the context for the fact, question, and answer generation task: {context}\n\n"
    prompt += "Extracted {num_questions} fact-question-answer pairs based on the given context using the following format: \n"
    prompt += "Fact: \n"
    prompt += "Question: \n"
    prompt += "Answer: \n\n"
    Prompt += "Only return the fact, question, and answer you generated. Do not include any additional information."
    return prompt

In [31]:
with open('Annotation/example.json', 'r') as f:
    examples = json.load(f)
prompt = prompt_generation(examples, chunks_content[0], 3)
print(prompt)

You are an AI assistant trained for data annotation.
Your task is to generate **fact-question-answer pairs** based on the given factual context. You will be given a context passage, and you will select a fact from the context, then ask a question from it, and then provide the answer to the asked question based on the selected fact. Ensure the questions are well-formed, unambiguous, and directly answerable using the provided context. Avoid speculative, open-ended questions, or generate any new context. These are some examples with questions and answers as well as the fact that help answer that question: 
Fact: The first known European to enter the region was the French explorer Robert de La Salle from Quebec during his 1669 expedition down the Ohio River.
Question: Who was the first known person to enter the Pittsburgh?
Answer: The first person to enter the Pittsburgh was the French explorer Robert de La Salle

Fact: In 1859, the Clinton and Soho iron furnaces introduced coke-fire smelt

In [32]:
results = pipe(prompt)

In [35]:
results[0]['generated_text']

"You are an AI assistant trained for data annotation.\nYour task is to generate **fact-question-answer pairs** based on the given factual context. You will be given a context passage, and you will select a fact from the context, then ask a question from it, and then provide the answer to the asked question based on the selected fact. Ensure the questions are well-formed, unambiguous, and directly answerable using the provided context. Avoid speculative, open-ended questions, or generate any new context. These are some examples with questions and answers as well as the fact that help answer that question: \nFact: The first known European to enter the region was the French explorer Robert de La Salle from Quebec during his 1669 expedition down the Ohio River.\nQuestion: Who was the first known person to enter the Pittsburgh?\nAnswer: The first person to enter the Pittsburgh was the French explorer Robert de La Salle\n\nFact: In 1859, the Clinton and Soho iron furnaces introduced coke-fir

In [4]:
file_path = "raw_data/citypittsburgh/citypittsburgh_conbined.txt"
with open(file_path, "r") as f:
    data = f.readlines()
print(data[0])

## ADA-Disability-Rights



In [ ]:
documents = SimpleDirectoryReader(
    input_dir="raw_data/description_pages", 
    required_exts=[".txt"],  # Ensure only .txt files are read
    recursive=True  # Enable recursive search inside subdirectories
).load_data()

3

In [ ]:
embed_model = HuggingFaceEmbedding(model_name='all-mpnet-base-v2')
llm = HuggingFaceLLM(model_name = )

In [ ]:
# Load the document(s)
documents = SimpleDirectoryReader("your_folder_path").load_data()

# Use an open-source embedding model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load an open-source LLM for Q&A generation (Mistral-7B or smaller)
llm = HuggingFaceLLM(
    model_name="mistralai/Mistral-7B-Instruct",
    tokenizer_name="mistralai/Mistral-7B-Instruct",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16}  # Efficient inference
)

# Create service context
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

# Build the index
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine = index.as_query_engine()
